In [2]:
%load_ext autoreload
%autoreload 2

1. sharing internet on the laocal machine

````
    ifconfig
````

2. scanning the local network to find the raspberry π

````
    nmap -T4 -sP 192.168.2.0/24
````

3 . cloned the repository on the π

````
  686  pip install pyserial
  687  vim .ssh/id_rsa.pub
  688  vim .ssh/id_rsa
  689  chmod 0600 .ssh/id_rsa
  690  git clone git@git.framasoft.org:laurentperrinet/elasticte.git
  691  cd elasticte/
  692  ls
  693  cat scenario_line_contraint.py
  694  python scenario_line_contraint.py  serial
  695  pip install -e .
  696  pip install --user -e .
  697  python scenario_line_contraint.py  serial
````


## git


In [3]:
!git s
!git add 2015-10-27\ élasticité\ r*

 M "2015-10-27 \303\251lasticit\303\251 expansion.ipynb"
A  "2015-10-27 \303\251lasticit\303\251 rapsberry pyserial.ipynb"
 M ../src/elasticite.py
?? ../.ipynb_checkpoints/
?? ../PLAN_PRINCIPE_PROTO/
?? ../Presentation-Dossier-Artistique/
?? ../TESTS_TRAME/
?? ../__temp_ipython__.png
?? ../files/elastic_fresnel.mp4
?? ../files/elastic_fresnel_gauche.mp4
?? ../files/elastic_fresnel_lateral.mp4
?? ../files/elastic_fresnel_pasapas.mp4
?? ../files/elastic_fresnel_rot.mp4
?? ../files/fresnel.mp4
?? "../files/grille_carr\303\251e.mp4"
?? ../files/lames_contraintes.mp4
?? ../files/lames_oscillantes.mp4
?? ../files/lames_oscillantes_fps5.mp4
?? "../notes-Laurent/2015-10-22_\303\251lasticit\303\251 LAMES-Dynamique_pdf.pdf"
?? .ipynb_checkpoints/
?? "2015-10-27 \303\251lasticit\303\251 expansion-r\303\251action diffusion.ipynb"
?? __temp__.mp4
?? __temp_ipython__.png
?? anim.gif
?? ../screenshot.png
?? ../src/elasticite.egg-info/


In [4]:
!git commit -am' on connecte le π '

[master 9275c64]  on connecte le π
 3 files changed, 289 insertions(+), 67 deletions(-)
 create mode 100644 "posts/2015-10-27 \303\251lasticit\303\251 rapsberry pyserial.ipynb"


In [5]:
! git push


Counting objects: 7, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 175.73 KiB | 0 bytes/s, done.
Total 7 (delta 5), reused 0 (delta 0)
To git@git.framasoft.org:laurentperrinet/elasticte.git
   ff8f33a..9275c64  master -> master
